In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

/Users/zurfarosa/demres/env/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [30]:
window = '12_to_7' #options: '12_to_7','10_to_5','8_to_3'

In [31]:
# pt_features = pd.read_csv('data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [32]:
pt_features = pd.read_csv('data/processed_data/pt_features_demins_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )



In [33]:
len(pt_features[pt_features['isCase']==True])

7973

In [34]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,age_at_index_date,CLOMETHIAZOLE_100_pdds,DIAZEPAM_100_pdds,FLURAZEPAM_100_pdds,FLURAZEPAM HYDROCHLORIDE_100_pdds,LOPRAZOLAM_100_pdds,LOPRAZOLAM MESILATELORAZEPAM_100_pdds,LORMETAZEPAM_100_pdds,MELATONIN_100_pdds,NITRAZEPAM_100_pdds,OXAZEPAM_100_pdds,TEMAZEPAM_100_pdds,ZALEPLON_100_pdds,ZOLPIDEM TARTRATE_100_pdds,ZOLPIDEM _100_pdds,ZOPICLONE_100_pdds,insomnia,insomnia_no_hypnotics,stroke,heart_failure,mental_illness,sleep_apnoea,chronic_pulmonary_disease,epilepsy,hypnotics_100_pdds,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,hypnotic_pdds:00000,hypnotic_pdds:00001_10,hypnotic_pdds:00011_100,hypnotic_pdds:00101_1000,hypnotic_pdds:01001_10000,hypnotic_pdds:10000_and_above
13349,3474249,33,249,1,2008-12-17,True,6578.0,2010-06-09,1992-01-31,35595,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6093,12647027,31,27,1,2010-01-26,False,NaN,2013-05-13,1997-11-28,12801,79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11556,7899258,20,258,1,2011-01-19,True,1916.0,2013-04-25,1996-03-31,37322,91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9376,4892242,20,242,1,2010-05-26,False,NaN,2013-05-03,1997-01-28,52332,90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10174,5236156,26,156,0,2008-08-04,False,NaN,2013-01-21,1996-07-18,61381,82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [35]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'age_at_index_date', 'CLOMETHIAZOLE_100_pdds', 'DIAZEPAM_100_pdds',
       'FLURAZEPAM_100_pdds', 'FLURAZEPAM HYDROCHLORIDE_100_pdds',
       'LOPRAZOLAM_100_pdds', 'LOPRAZOLAM MESILATELORAZEPAM_100_pdds',
       'LORMETAZEPAM_100_pdds', 'MELATONIN_100_pdds', 'NITRAZEPAM_100_pdds',
       'OXAZEPAM_100_pdds', 'TEMAZEPAM_100_pdds', 'ZALEPLON_100_pdds',
       'ZOLPIDEM TARTRATE_100_pdds', 'ZOLPIDEM _100_pdds',
       'ZOPICLONE_100_pdds', 'insomnia', 'insomnia_no_hypnotics', 'stroke',
       'heart_failure', 'mental_illness', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'hypnotics_100_pdds',
       'age_at_index_date:65-69', 'age_at_index_date:70-74',
       'age_at_index_date:75-79', 'age_at_index_date:80-84',
       'age_at_index_date:85-89', 'age_at_index_date:90-99',
       'age_at_index_date:above_99', 'hypnotic_pdds:0000

In [36]:
# characteristics = [column for column in pt_features.columns if column not in [
#     'patid',  'yob', 'pracid', 'index_date', 'isCase',
#     'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [37]:
columns_for_inclusion = [
    'age_at_index_date',
    'female',
    'stroke',
    'heart_failure',
    'mental_illness',
    'sleep_apnoea', 
    'chronic_pulmonary_disease', 
    'epilepsy',
    'hypnotics_100_pdds',
    'insomnia',
    'age_at_index_date:65-69', 'age_at_index_date:70-74',
    'age_at_index_date:75-79', 'age_at_index_date:80-84',
    'age_at_index_date:85-89', 'age_at_index_date:90-99',
    'age_at_index_date:above_99', 
    'hypnotics_100_pdds','hypnotic_pdds:00000',
    'hypnotic_pdds:00001_10', 'hypnotic_pdds:00011_100',
    'hypnotic_pdds:00101_1000', 'hypnotic_pdds:01001_10000',
    'hypnotic_pdds:10000_and_above'
]

In [38]:
baselines = add_baseline_characteristics(columns_for_inclusion,pt_features)

In [39]:
len(pt_features[pt_features['isCase']==False]),len(pt_features[pt_features['isCase']==True])

(7973, 7973)

In [40]:
baselines[['Cases','Controls']]

,Cases,Controls
female,5353 (67.1%),5353 (67.1%)
stroke,949 (11.9%),629 (7.9%)
heart_failure,515 (6.5%),472 (5.9%)
mental_illness,3270 (41.0%),2243 (28.1%)
sleep_apnoea,17 (0.2%),5 (0.1%)
chronic_pulmonary_disease,2059 (25.8%),1478 (18.5%)
epilepsy,166 (2.1%),102 (1.3%)
insomnia,763 (9.6%),550 (6.9%)
age_at_index_date:65-69,275 (3.4%),275 (3.4%)
age_at_index_date:70-74,678 (8.5%),678 (8.5%)


In [41]:
len(pt_features[(pt_features['insomnia']==False) & (pt_features['isCase']==False)])

7423